In [1]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from pandas import json_normalize
from src.api_functions import *
from src.cleaning_functions import *
import requests
from bs4 import BeautifulSoup
import lxml.html as lh
from src.visual_functions import *
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from pylab import mpl

In [2]:
###--------------------------------------------API_adquisition--------------------------------------------------###
url_world = "https://world-population.p.rapidapi.com/worldpopulation"
world_pop = request_world_population(url_world)
format_json(world_pop) #Returns global population in .json

countries_name = []
url_countries = "https://world-population.p.rapidapi.com/allcountriesname"
test = request_country_names(url_countries)
dict_countries = format_json(test) #Returns a list of dicts of countries in .json
a = dict_countries.get('body')
list_countries = []
for i in a.values():
    list_countries.append(i) 
final_list = [item for sublist in list_countries for item in sublist] #Returns a list of countries

url_specific = "https://world-population.p.rapidapi.com/population"
database_api = request_all_countries_population(url_specific, final_list)
df1 = pd.DataFrame(database_api)
df1.rename(columns={"country_name": "country"}, inplace=True)

'''---------------------------------------------------------------------------------------------------------------'''
df1.to_csv('data/population-data.csv' , index = False)
'''---------------------------------------------------------------------------------------------------------------'''

'---------------------------------------------------------------------------------------------------------------'

In [3]:
###---------------------------------------Database_csv_managing--------------------------------------###

# df1 = dataframe from API
# df2 = dataframe from energy .csv
# df3 = dataframe from merging df1 and a modified version of df2
# df4 = dataframe derived from df3 containing only the year 2020
# df5 = dataframe derived from df4 without NaNs and with grouped columns by categories
# df6 = dataframe from avg-rate .csv (population)

csv0 = 'data/population-data.csv'
csv1 = 'data/energy-data/owid-energy-data.csv'
csv2 = 'data/avg-rate.csv'
df1 = load_csv(csv0)
df2 = load_csv(csv1)

In [4]:
list_1 = df1['country'] # df1 --> population
list_2 = df2['country'] # df2 --> energy
exclusion_list = np.setdiff1d(list_1,list_2)
df_ec = df2[df2['country'].isin(exclusion_list)] # Fork here, for correcting the prediction (in a future update)
df2_mod = df2[~df2['country'].isin(exclusion_list)]
df1.set_index('country', inplace=True)
df2_mod.set_index('country', inplace=True)
df3 = df2_mod.merge(df1, how='inner', on='country') # We combine both datasets through the common countries. The
df3.isna().sum()
df3.dropna(axis=1, how="all", inplace=True) # We don't use the function here because it must be applied to the columns
df3.drop(["iso_code","carbon_intensity_elec"], axis=1, inplace=True)
'''---------------------------------------------------------------------------------------------------------------'''
df3.to_csv('data/energy_ranged_with_pop',index = True)
'''---------------------------------------------------------------------------------------------------------------'''

'---------------------------------------------------------------------------------------------------------------'

In [5]:
df6 = load_csv(csv2)
df6.drop(['LocID', 'VarID', 'Variant', 'Time', 'TFR',
       'NRR', 'CBR', 'Births', 'LEx', 'LExMale', 'LExFemale', 'IMR', 'Q5',
       'CDR', 'Deaths', 'DeathsMale', 'DeathsFemale', 'CNMR', 'NetMigrations',
          'NatIncr', 'SRB', 'MAC'], axis=1, inplace=True)

df6.set_index('MidPeriod',drop=True,inplace=True)
df6.rename(columns={"Location":"country"}, inplace=True)
df6.groupby(['country'],as_index = True).mean()

,GrowthRate
country,
Afghanistan,1.512467
Africa,1.955567
African Group,1.955967
African Union,1.956133
African Union: Central Africa,2.149567
...,...
World Bank Regional Groups (developing only),1.098833
World Bank income groups,NaN
Yemen,1.622800


In [6]:
csv3 = 'data/energy_ranged_with_pop'
df3 = load_csv(csv3)


df7 = df3.merge(df6, how='left', on='country')

In [7]:
df7.dropna(axis=1, how='all') # We get the interannual variation rate of population (per country)

,country,year,coal_prod_change_pct,coal_prod_change_twh,gas_prod_change_pct,gas_prod_change_twh,oil_prod_change_pct,oil_prod_change_twh,energy_cons_change_pct,energy_cons_change_twh,...,wind_cons_change_pct,wind_share_energy,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_energy_per_capita,population_y,ranking,world_share,GrowthRate
0,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,38928346,37,0.499387,1.296
1,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,38928346,37,0.499387,1.683
2,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,38928346,37,0.499387,2.026
3,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,38928346,37,0.499387,2.307
4,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,38928346,37,0.499387,2.544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399304,Zimbabwe,2019,-35.015,-9.422,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,14862924,74,0.190667,0.441
399305,Zimbabwe,2019,-35.015,-9.422,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,14862924,74,0.190667,0.310
399306,Zimbabwe,2019,-35.015,-9.422,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,14862924,74,0.190667,0.191
399307,Zimbabwe,2019,-35.015,-9.422,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,14862924,74,0.190667,0.098


In [8]:
'''---------------------------------------------------------------------------------------------------------------'''
df7.to_csv('data/pop_avg_rate.csv' , index = True)
'''---------------------------------------------------------------------------------------------------------------'''

'---------------------------------------------------------------------------------------------------------------'

In [9]:
url='https://www.antipodas.net/coordenadaspais/'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
[len(C) for (title,C) in col]
Dict={title:column for (title,column) in col}
coordinates=pd.DataFrame(Dict)
coordinates = coordinates.drop(['Nº de Habitantes'],axis=1)
coordinates = coordinates.rename(columns={'País':'country','Coordenadas Geográficas':'coordinates'})

1:"País"
2:"Coordenadas Geográficas"
3:"Nº de Habitantes"


In [10]:
csv3 = 'data/energy_ranged_with_pop'
df3 = load_csv(csv3)

list_1 = df3['country']
list_2 = coordinates['country']
exclusion_list = np.setdiff1d(list_2,list_1)
coordinates_mod = coordinates[~coordinates['country'].isin(exclusion_list)]
df3.set_index('country', inplace=True)
coordinates_mod.set_index('country', inplace=True)
dfinal = df3.merge(coordinates_mod, how='inner', on='country') # We combine both datasets through the common countries. The
# uncommon countries impact is estimated apart
dfinal_coords = dfinal['coordinates'].str.split(expand=True)
'''---------------------------------------------------------------------------------------------------------------'''
dfinal_coords.to_csv('output/dfinal_coords.csv' , index = True)
dfinal.to_csv('output/dfinal.csv' , index = True)
'''---------------------------------------------------------------------------------------------------------------'''

'---------------------------------------------------------------------------------------------------------------'

In [11]:
df3 = load_csv('data/energy_ranged_with_pop')
df4 = df3[df3['year']==2020]

In [12]:
filter_dataframe(df3,coal')

SyntaxError: EOL while scanning string literal (<ipython-input-12-56e627c52ddd>, line 1)

In [ ]:
#new_dataframe(df,category,string)


tc_coal_info = columns_extractor(df4,'coal')
th_gas_info = columns_extractor(df3,'gas')
tc_gas_info = columns_extractor(df4,'gas')
th_oil_info = columns_extractor(df3,'oil')
tc_oil_info = columns_extractor(df4,'oil')
th_nuclear_info = columns_extractor(df3,'nuclear')
tc_nuclear_info = columns_extractor(df4,'nuclear')
th_biofuel_info = columns_extractor(df3,'biofuel')
tc_biofuel_info = columns_extractor(df4,'biofuel')

th_hydro_info = columns_extractor(df3,'hydro')
tc_hydro_info = columns_extractor(df4,'hydro')
th_solar_info = columns_extractor(df3,'solar')
tc_solar_info = columns_extractor(df4,'solar')
th_wind_info = columns_extractor(df3,'wind')
tc_wind_info = columns_extractor(df4,'wind')

In [ ]:
'''---------------------------------------------------------------------------------------------------------------'''
save_table_csv(th_coal_info)
save_table_csv(th_coal_info)
save_table_csv(th_gas_info)
save_table_csv(th_gas_info)
save_table_csv(th_oil_info)
save_table_csv(th_oil_info)
save_table_csv(th_nuclear_info)
save_table_csv(th_nuclear_info)
save_table_csv(th_biofuel_info)
save_table_csv(th_biofuel_info)
save_table_csv(th_hydro_info)
save_table_csv(th_hydro_info)
save_table_csv(th_solar_info)
save_table_csv(th_solar_info)
save_table_csv(th_wind_info)
save_table_csv(th_wind_info)
'''---------------------------------------------------------------------------------------------------------------'''

In [ ]:
###--------------------------------------------Storytelling--------------------------------------------------###

In [ ]:
x = df3['population']

In [ ]:
y = np.multiply(df3['population'],df7['GrowthRate'])

In [ ]:
parameter = liner_fitting(x,y)
draw_data = calculate_lin(x,parameter[0],parameter[1])
draw_lin(x,draw_data,y)